<a href="https://colab.research.google.com/github/Ishwarya-2004/Fashion-trend-forecasting/blob/main/Fashion_Trend_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
import datetime
import re
from collections import Counter
from difflib import get_close_matches
import praw
import gradio as gr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import os
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
from statsmodels.tsa.statespace.sarimax import SARIMAX
from textblob import TextBlob

# Load Dataset
file_path = "Fashion_Retail_Sales_Modified.csv"
df = pd.read_csv(file_path)
df["Date Purchase"] = pd.to_datetime(df["Date Purchase"], errors="coerce")
df["Month"] = df["Date Purchase"].dt.to_period("M").dt.to_timestamp()
unique_countries = df["Country"].dropna().unique()

# Reddit API
reddit = praw.Reddit(
    client_id="",
    client_secret="",
    user_agent="fashion-trend-app"
)

# Unsplash API
UNSPLASH_ACCESS_KEY = ""
IMAGE_DIR = "fashion_images"
if not os.path.exists(IMAGE_DIR):
    os.makedirs(IMAGE_DIR)

# Load Pretrained ResNet Model
model = models.resnet50(pretrained=True)
model.eval()

def preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])
    image = Image.open(image_path).convert("RGB")
    return transform(image).unsqueeze(0)

def extract_features(image_path):
    image_tensor = preprocess_image(image_path)
    with torch.no_grad():
        features = model(image_tensor)
    return features.numpy().flatten()

def fetch_fashion_images(product, num_images=5):
    url = f"https://api.unsplash.com/search/photos?query={product}&per_page={num_images}&client_id={UNSPLASH_ACCESS_KEY}"
    response = requests.get(url)
    if response.status_code != 200:
        return ["Error fetching images from Unsplash."]
    data = response.json()
    image_paths = []
    for i, item in enumerate(data["results"]):
        img_url = item["urls"]["regular"]
        img_path = os.path.join(IMAGE_DIR, f"{product.replace(' ', '_')}_{i}.jpg")
        img_data = requests.get(img_url).content
        with open(img_path, "wb") as handler:
            handler.write(img_data)
        image_paths.append(img_path)
    return image_paths

def select_best_image(product):
    image_paths = fetch_fashion_images(product, num_images=5)
    if not image_paths:
        return None
    feature_vectors = np.array([extract_features(img) for img in image_paths])
    distances = np.linalg.norm(feature_vectors - feature_vectors.mean(axis=0), axis=1)
    best_idx = np.argmin(distances)
    return image_paths[best_idx]

def get_products(selected_country):
    if selected_country in unique_countries:
        return df[df["Country"] == selected_country]["Item Purchased"].dropna().unique().tolist()
    return []

month_festivals = {
    "January": ["London Textile Fair (UK)", "Berlin Fashion Week (Germany)"],
    "February": ["New York Fashion Week (USA)", "London Fashion Week (UK)"],
    "March": ["Los Angeles Fashion Week (USA)"],
    "April": ["Coachella Festival Fashion Trends (USA)"],
    "May": ["Met Gala (USA)"],
    "June": ["Royal Ascot Fashion Event (UK)"],
    "July": ["Miami Swim Week (USA)", "Berlin Fashion Week (Germany)"],
    "August": ["Bread & Butter by Zalando (Germany)"],
    "September": ["New York Fashion Week (USA)", "London Fashion Week (UK)"],
    "October": ["Los Angeles Fashion Week (USA)"],
    "November": ["Black Friday Sales (USA, UK, Germany)"],
    "December": ["Christmas Fashion Markets (Global)", "New Year Shopping Events (Global)"]
}

def get_upcoming_festivals(date):
    try:
        date = pd.to_datetime(date, errors='coerce')
        if pd.isna(date):
            return "Invalid date format."
        months = [(date.month + i - 1) % 12 + 1 for i in range(3)]
        festival_list = []
        for m in months:
            month_name = pd.to_datetime(f"2025-{m}-01").strftime("%B")
            festival_list.extend(month_festivals.get(month_name, []))
        return ", ".join(festival_list) if festival_list else "No festivals found."
    except:
        return "Error fetching festivals."

# Get trending fashion keywords from Reddit
from fashion_trends import fashion_trends

# Function to get trending fashion keywords for a given month
def get_trending_fashion_keywords(month):
    return fashion_trends.get(month.lower().strip(), [])

# Example usage
def display_fashion_trends(month):
    trends = get_trending_fashion_keywords(month)
    if trends:
        print(f"Trending fashion for {month.capitalize()}:")
        for trend in trends:
            print(f"- {trend}")
    else:
        print(f"No fashion trends available for {month.capitalize()}.")

# Input the month you want to check trends for
month_input = input("Enter a month: ")
display_fashion_trends(month_input)
# Sentiment analysis from Reddit comments
def get_sentiment_pie(product):
    subreddit = reddit.subreddit("fashion")
    sentiments = {"positive": 0, "neutral": 0, "negative": 0}
    for post in subreddit.search(product, sort="relevance", limit=10):
        post.comments.replace_more(limit=0)
        for comment in post.comments[:20]:
            blob = TextBlob(comment.body)
            polarity = blob.sentiment.polarity
            if polarity > 0.1:
                sentiments["positive"] += 1
            elif polarity < -0.1:
                sentiments["negative"] += 1
            else:
                sentiments["neutral"] += 1
    total = sum(sentiments.values())
    if total == 0:
        sentiments = {"positive": 33, "neutral": 34, "negative": 33}
    else:
        for key in sentiments:
            sentiments[key] = (sentiments[key] / total) * 100

    fig, ax = plt.subplots()
    ax.pie(sentiments.values(), labels=sentiments.keys(), autopct='%1.1f%%',
           startangle=90, colors=["#4CAF50", "#FFC107", "#F44336"])
    ax.set_title("Fashion Sentiment Analysis")
    plt.savefig("sentiment_pie_chart.png")
    plt.close()
    return "sentiment_pie_chart.png"

def forecast_sales(selected_country, selected_product, future_date, future_units):
    if selected_country not in unique_countries:
        return "Invalid Country", None, None, [], None
    country_data = df[df["Country"] == selected_country]
    if selected_product not in country_data["Item Purchased"].dropna().unique():
        return "Invalid Product", None, None, [], None

    product_data = country_data[country_data["Item Purchased"] == selected_product]
    product_data = product_data.groupby("Month")["Purchase Amount (USD)"].sum().reset_index().sort_values("Month")
    try:
        model = SARIMAX(product_data["Purchase Amount (USD)"], order=(1,1,1), seasonal_order=(1,1,1,12)).fit()
        forecast_steps = 36
        forecast = model.get_forecast(steps=forecast_steps).predicted_mean
        forecast_dates = pd.date_range(start=product_data["Month"].max(), periods=forecast_steps + 1, freq="M")[1:]

        future_date_parsed = pd.to_datetime(future_date, errors="coerce")
        if future_date_parsed is pd.NaT or not (forecast_dates[0] <= future_date_parsed <= forecast_dates[-1]):
            return f"Error: Date out of range ({forecast_dates[0].strftime('%Y-%m')} to {forecast_dates[-1].strftime('%Y-%m')})", None, None, [], None

        closest_date = min(forecast_dates, key=lambda x: abs(x - future_date_parsed))
        revenue = (forecast.iloc[list(forecast_dates).index(closest_date)] / max(product_data["Purchase Amount (USD)"].mean(), 1)) * future_units

        pie_path = get_sentiment_pie(selected_product)
        festivals = get_upcoming_festivals(future_date)
        month_name = pd.to_datetime(future_date).strftime("%B")
        trending_products = get_trending_fashion_keywords(month_name)

        if selected_product.lower() in trending_products:
            trend_msg = "✅ Your selected product is in trend!"
            trending_image_paths = []
        else:
            trend_msg = f"❌ Your selected product is not in trend.\nTrending Products for {month_name}: {', '.join(trending_products)}"
            trending_image_paths = [select_best_image(p) for p in trending_products]

        return (
            f"Predicted Revenue: ${revenue:.2f}\nFestive Seasons: {festivals}\n{trend_msg}",
            pie_path,
            [select_best_image(selected_product)],
            trending_image_paths,
            f"Trending Products for {month_name}"
        )
    except Exception as e:
        return f"Error: {str(e)}", None, None, [], None

# Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("## 🛍️ Fashion Trend & Sales Forecasting App")

    selected_country = gr.Dropdown(choices=unique_countries.tolist(), label="🌍 Select Country")
    selected_product = gr.Dropdown(choices=[], label="👗 Select Product")
    selected_country.change(fn=lambda x: gr.update(choices=get_products(x)), inputs=[selected_country], outputs=[selected_product])

    future_date = gr.Textbox(label="📅 Enter Future Sale Date (YYYY-MM)")
    future_units = gr.Number(label="📦 Expected Units Sold")
    submit_button = gr.Button("🔍 Predict & Get Trends")

    output_text = gr.Textbox(label="📈 Forecast Summary & Trend Info")
    output_image = gr.Image(label="📊 Sentiment Analysis Pie Chart")
    output_gallery = gr.Gallery(label="🖼️ Selected Product Image")
    trending_heading = gr.Textbox(visible=False)
    trending_gallery = gr.Gallery(label="🌟 Trending Product Images")

    feedback = gr.Radio(["👍 Good", "👎 Needs Improvement"], label="💬 Your Feedback")

    submit_button.click(
        fn=forecast_sales,
        inputs=[selected_country, selected_product, future_date, future_units],
        outputs=[output_text, output_image, output_gallery, trending_gallery, trending_heading]
    )

demo.launch(share=True)
